In [ ]:
mn1,mn2,mx1,mx2=55.655155181884766, 37.49541473388672, 55.753211975097656, 37.75920104980469

In [ ]:
import numpy as np
import pandas as pd
range_lat=np.arange(55.655155, 55.753211975097656+0.0028016226632254465, 0.0028016226632254465)
range_long=np.arange(37.49541473388672, 37.75920104980469+0.007536751883370536, 0.007536751883370536)

In [ ]:
!pip install  overpy

     |████████████████████████████████| 51kB 2.9MB/s 
  Created wheel for overpy: filename=overpy-0.6-cp37-none-any.whl size=14107 sha256=38c6c7fbfc1a800bc0853165da555bebf25bb95e18613c7a3b1675b20962bb84
  Stored in directory: /root/.cache/pip/wheels/bb/84/dc/059d81d88c256d439283d61be933a88937f28fa1eb49f03876
Successfully built overpy


In [ ]:
import overpy
api = overpy.Overpass()

def getarea(lat, lon, latend, lonend):
    coord = f"{lat},{lon},{latend},{lonend}"
    #[out:json];
    string = f"""node({coord}); out;
    """

    data = api.query(string)
    return data.nodes

#https://wiki.openstreetmap.org/wiki/RU:%D0%9A%D0%B0%D0%BA_%D0%BE%D0%B1%D0%BE%D0%B7%D0%BD%D0%B0%D1%87%D0%B8%D1%82%D1%8C
# Пример вывода данных
# zone = getarea()
# for node in zone:
#     if 'amenity' in node.tags:
#         print(f"amen= {node.tags['amenity']}")
#     if 'shop' in node.tags:
#         print(f"shop= {node.tags['shop']}")
#     print(f"lon= {node.lon} lat={node.lat} ")

In [ ]:
shift_lat=range_lat[1]-range_lat[0]
shift_long=range_long[1]-range_long[0]
import time
area_res=getarea(mn1,mn2,mx1,mx2)

In [ ]:
def OSMGetData(area_res, lat, lon, latend, longend, amensp = set(), shopsp = set(), buildsp = set(), leisusp = set(), craftsp = set()):
  amenity_specs = amensp
  buildings_specs = buildsp
  shop_specs = shopsp
  leisure_specs = leisusp
  craft_specs=craftsp
  res=[]
  for i in area_res:
    f=0
    if (lat<=i.lat<=latend and lon<=i.lon<=longend):
      if 'amenity' in i.tags.keys():
        if len(set([i.tags['amenity']]).intersection(amenity_specs))>0:
          res.append(i)
          f=1
      elif not f and 'building' in i.tags.keys():
        if len(buildings_specs.intersection(set([i.tags['building']])))>0:
          res.append(i)
          f=1
      elif not f and 'shop' in i.tags.keys():
        if len(shop_specs.intersection(set([i.tags['shop']])))>0:
          res.append(i)
          f=1
      elif not f and 'leisure' in i.tags.keys():
        if len(leisure_specs.intersection(set([i.tags['leisure']])))>0:
          res.append(i)
          f=1
      elif not f and 'craft' in i.tags.keys():
        if len(craft_specs.intersection(set([i.tags['craft']])))>0:
          res.append(i)
          f=1
  return res

In [ ]:
data=pd.read_csv('data_with_metro.csv')
data=data.drop(['Unnamed: 0'], axis=1)

In [ ]:
data.head()

,lat,long,people,weekend_mean,workdays_mean,day_mean,night_mean,metro,metro_dist
0,"(55.655, 55.658]","(37.495, 37.503]",0.0,0.0,0.0,0.0,0.0,Юго-западная,0.016882
1,"(55.655, 55.658]","(37.503, 37.51]",0.0,0.0,0.0,0.0,0.0,Проспект Вернадского,0.019732
2,"(55.655, 55.658]","(37.51, 37.518]",0.0,0.0,0.0,0.0,0.0,Беляево,0.018684
3,"(55.655, 55.658]","(37.518, 37.526]",0.0,0.0,0.0,0.0,0.0,Беляево,0.013970
4,"(55.655, 55.658]","(37.526, 37.533]",0.0,0.0,0.0,0.0,0.0,Калужская,0.009766


In [ ]:
newdata=pd.DataFrame(np.zeros(data.shape[0]))

In [ ]:
data111={'butcher':[],'bakery':[],'shop':[], 'cafe':[],
         'drinking':[], 'clinic':[], 'vets':[], 'dentist':[],
         'chemist':[], 'pharmacy':[], 'florist':[], 'hairdresser':[],
         'spa':[],'fixer':[],'photo':[], 'tobacco':[]}

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
for i in tqdm_notebook(range(data.shape[0])):
  a=list(map(float,data.lat[i][1:-1].split(',')))
  b=list(map(float,data.long[i][1:-1].split(',')))
  latc=a[0]
  latend=a[1]
  longc=b[0]
  longend=b[1]
  # ФУД РИТЕЙЛ 
  # Мясо
  sh_prm = {'butcher'}
  data111['butcher'].append(len(OSMGetData(area_res,latc,longc,latend,longend, shopsp = sh_prm)))
  # Булочная
  sh_prm = {'bakery'}
  data111['bakery'].append(len(OSMGetData(area_res,latc,longc,latend,longend, shopsp = sh_prm)))
  # Остальное
  sh_prm = {'supermarket', 'convenience'}
  bld_prm = {'store', 'retail'}
  data111['shop'].append(len(OSMGetData(area_res,latc,longc,latend,longend, shopsp = sh_prm)))

  # РЕСТОРАНЫ
  # Кафе/Рестораны
  am_prm = {'cafe', 'restaurant','fast_food', 'food_court'}
  data111['cafe'].append(len(OSMGetData(area_res,latc,longc,latend,longend, am_prm)))
  # Клубы/Бары
  am_prm = {'bar', 'biergarten',  'pub', 'nightclub', 'stripclub'}
  data111['drinking'].append(len(OSMGetData(area_res,latc,longc,latend,longend, am_prm, sh_prm, bld_prm)))

  # МЕД УСЛУГИ
  # Клиника
  am_prm = {'clinic'}
  data111['clinic'].append(len(OSMGetData(area_res,latc,longc,latend,longend, am_prm)))
  # Вет клиника
  am_prm = {'veterinary'}
  data111['vets'].append(len(OSMGetData(area_res,latc,longc,latend,longend, am_prm)))
  # Стоматология
  am_prm = {'dentist'}
  data111['dentist'].append(len(OSMGetData(area_res,latc,longc,latend,longend, am_prm)))

  # МАГАЗИНЫ
  # Товары для дома
  sh_prm = {'houseware', 'cosmetics', 'chemist', 'stationery'}
  data111['chemist'].append(len(OSMGetData(area_res,latc,longc,latend,longend, shopsp = sh_prm)))
  # Аптека
  am_prm = {'pharmacy'}
  data111['pharmacy'].append(len(OSMGetData(area_res,latc,longc,latend,longend, am_prm)))
  # Цветочный
  sh_prm = {'florist'}
  data111['florist'].append(len(OSMGetData(area_res,latc,longc,latend,longend, shopsp=sh_prm)))

  # САЛОНЫ
  # Парикмахерская
  sh_prm = {'hairdresser'}
  data111['hairdresser'].append(len(OSMGetData(area_res,latc,longc,latend,longend, shopsp = sh_prm)))
  # Салон красоты
  am_prm ={'sauna'}
  sh_prm = {'beauty','massage'}
  le_prm={'sauna','tanning_salon'}
  data111['spa'].append(len(OSMGetData(area_res,latc,longc,latend,longend, am_prm, sh_prm, leisusp = le_prm)))

  # УСЛУГИ + ТОРГОВЛЯ
  # Ремонт
  craft_prm = {'watchmaker', 'electronics_repair'}
  data111['fixer'].append(len(OSMGetData(area_res,latc,longc,latend,longend, craftsp = craft_prm)))
  # Фотоателье
  sh_prm = {'photo', 'photo_studio'}
  craft_prm = {'photographer'}
  data111['photo'].append(len(OSMGetData(area_res,latc,longc,latend,longend, shopsp = sh_prm, craftsp = craft_prm)))
  # Табачка
  sh_prm = {'tobacco'}
  data111['tobacco'].append(len(OSMGetData(area_res,latc,longc,latend,longend, shopsp = sh_prm)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [1]:
data111

NameError: ignored

In [ ]:
import pickle

In [ ]:
with open('dump.pkl','wb') as f:
  pickle.dump(data111,f)